# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [10]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
import re
import numpy as np

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/mohammad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mohammad/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mohammad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# load data from database
engine = create_engine('sqlite:///ETL_final.db')
df = pd.read_sql_table('cleaned_table', engine)
                      
X = df['message']
y = df.iloc[:,4:]
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [12]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

wnl = WordNetLemmatizer()

def tokenize(text):
    cleaned_text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    cleaned_text = word_tokenize(cleaned_text)
    cleaned_text = [w for w in cleaned_text if w not in stopwords.words("english")]
    return [wnl.lemmatize(x) for x in cleaned_text]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier

class MyTfidfVectorizer(TfidfVectorizer):
    def fit_transform(self, X, y):
        result = super(MyTfidfVectorizer, self).fit_transform(X, y)
        result.sort_indices()
        return result

pipeline = Pipeline([
    ('tfidf_vectorizer', MyTfidfVectorizer(tokenizer=tokenize)),
    ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X,y)

model = pipeline.fit(X_train, y_train)

In [15]:
y_preds = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
from sklearn.metrics import accuracy_score, classification_report, precision_score

y_test_numpy = y_test.to_numpy()

for i, column_name in zip(range(36), y_test.columns):
    print(f'''Column_name: {column_name}
    {classification_report(y_test_numpy[:,i], y_preds[:,i])}
    --------------------------------------------------------------------------''')
    
print(f'''Aggregated report
    {classification_report(y_test_numpy.flatten(), y_preds.flatten())}''')

Column_name: related
                  precision    recall  f1-score   support

           0       0.65      0.12      0.20      1509
           1       0.78      0.98      0.87      4995
           2       0.33      0.10      0.15        50

    accuracy                           0.77      6554
   macro avg       0.59      0.40      0.41      6554
weighted avg       0.75      0.77      0.71      6554

    --------------------------------------------------------------------------
Column_name: request
                  precision    recall  f1-score   support

           0       0.91      0.97      0.94      5459
           1       0.76      0.50      0.60      1095

    accuracy                           0.89      6554
   macro avg       0.83      0.74      0.77      6554
weighted avg       0.88      0.89      0.88      6554

    --------------------------------------------------------------------------
Column_name: offer
                  precision    recall  f1-score   support

      

Column_name: transport
                  precision    recall  f1-score   support

           0       0.96      1.00      0.98      6243
           1       0.70      0.20      0.31       311

    accuracy                           0.96      6554
   macro avg       0.83      0.60      0.64      6554
weighted avg       0.95      0.96      0.95      6554

    --------------------------------------------------------------------------
Column_name: buildings
                  precision    recall  f1-score   support

           0       0.97      0.99      0.98      6231
           1       0.65      0.39      0.48       323

    accuracy                           0.96      6554
   macro avg       0.81      0.69      0.73      6554
weighted avg       0.95      0.96      0.95      6554

    --------------------------------------------------------------------------
Column_name: electricity
                  precision    recall  f1-score   support

           0       0.98      1.00      0.99      6


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'classifier__estimator__n_estimators': [50, 100, 200],
    'classifier__estimator__learning_rate':[.01, .1, 1]
}

cv = GridSearchCV(pipeline, param_grid=parameters) 

model_cv = cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
cv.best_estimator_

Pipeline(memory=None,
         steps=[('tfidf_vectorizer',
                 MyTfidfVectorizer(analyzer='word', binary=False,
                                   decode_error='strict',
                                   dtype=<class 'numpy.float64'>,
                                   encoding='utf-8', input='content',
                                   lowercase=True, max_df=1.0,
                                   max_features=None, min_df=1,
                                   ngram_range=(1, 1), norm='l2',
                                   preprocessor=None, smooth_idf=True,
                                   stop_words=None, strip_accents=None,
                                   sublinear_tf=False,
                                   token_pattern='(?u)\\b\\w\\w+\\b',
                                   tokenizer=<function tokenize at 0x7f4b4af80ae8>,
                                   use_idf=True, vocabulary=None)),
                ('classifier',
                 MultiOutputClassifie

In [ ]:
y_preds_cv = model_cv.predict(X_test)

In [ ]:
for i, column_name in zip(range(36), y_test.columns):
    print(f'''Column_name: {column_name}
    {classification_report(y_test_numpy[:,i], y_preds_cv[:,i])}
    --------------------------------------------------------------------------''')
    
print(f'''Aggregated report
    {classification_report(y_test_numpy.flatten(), y_preds_cv.flatten())}''')

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.